In [2]:
import pandas as pd
from utils import *
import numpy as np
from mapreduce import FilteringMapReduce, map_reduce_df

In [3]:
days = get_days_in_range('2017-08-01', '2017-08-15')
types = {'id': 'int32',
         'date': 'int16',
         'item_nbr': 'int32',
         'store_nbr': 'int16',
         'unit_sales': 'float32',
         'onpromotion': bool}
cols = ['id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion']
mapreduce = FilteringMapReduce(lambda df: df[df.date.isin(days)])

df_train = map_reduce_df('./data/train_processed.csv', mapreduce, types=types, position=120000000, cols=cols, verbose=True)

Reading batch from position 120000000, batch size 10000000...
Filtering 5497041...
Filtered 1570968, mapping...
Mapped, reducing...
Batch done.
End of dataset is found.


In [4]:
df_train.ix[df_train.unit_sales < 0, "unit_sales"] = 0
df_train["unit_sales"] = np.log1p(df_train["unit_sales"])

In [ ]:
u_dates = df_train.date.unique()
u_stores = df_train.store_nbr.unique()
u_items = df_train.item_nbr.unique()
df_train.set_index(["date", "store_nbr", "item_nbr"], inplace=True)
df_train = df_train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=["date", "store_nbr", "item_nbr"]
    )
)
df_train.loc[:, "unit_sales"].fillna(0, inplace=True)
df_train.loc[:, "onpromotion"].fillna(False, inplace=True)
df_train.reset_index(inplace=True)

In [ ]:
from utils import get_index_date
get_index_date(df_train['date'].min())

In [ ]:
from utils import get_index_date
get_index_date(df_train['date'].max())

In [ ]:
len(df_train)

In [ ]:
train = df_train.groupby(['item_nbr', 'store_nbr', 'onpromotion'])['unit_sales'].mean().to_frame('unit_sales')
train.head()

In [10]:
!telegram-send "Preparations finished"

In [11]:
test = pd.read_csv("./data/test.csv").set_index(['item_nbr', 'store_nbr', 'onpromotion']).join(df_train, how='left').fillna(0.0).reset_index()

In [42]:
test[['id', 'unit_sales']].to_csv('./submissions/kernel_mean_baseline4.csv', index=None)

In [44]:
!telegram-send "Submission is done"